In [1]:
import xarray as xr
xr.set_options(display_style="html")

from pathlib import Path
import scipy.io  as sio
import numpy as np
import pandas as pd
from matplotlib import animation as ani, pyplot as plt
import seaborn as sns


import pickle

In [2]:
def get_project_root() -> Path:
    """Return a base path of current folder, similar to R packege 'here'.
        
    Returns:
        Path: Path object pointing to the parent of the folder where the script is located.
            If your script is `/root/scripts/script.py`, the result will be an absolute path '/root'.
    """
    try:
        return Path(__file__).parent.parent
    except NameError as error:
        # This will give you  root folder of the project if this runned through jupyter notebook
        return Path().joinpath().absolute().parents[0]
    except Exception as exception:
        print(exception)
        
def load_conditions_coding(keys_path: Path) -> dict:
    """ Load coding telling us what conditions was representing specific recording (Sham or real TDCS)

    Args:
        keys_path (string): Path to file containing matlab struct with the simple mapping

    Returns:
        DataFrame: Dataframe cointaining two rows subject cond like Subject01_A or Subject01_B and condition "sham" or "real"
    """

    keys = sio.loadmat(keys_path)
    row_names = keys["keys"].tolist()[0]
    #row_names=  [l.flatten()[0] for l in lista.flatten()] #fl
    temp_df = pd.DataFrame()   
    temp_df["subject_cond"]  = [l[0].flatten()[0] for l in row_names]
    temp_df["subject_cond"] = temp_df['subject_cond'].astype('category').str.strip()
    temp_df["T"] =  [l[1].flatten()[0] for l in row_names]
    return temp_df


root= get_project_root()
coding_path = root / 'data' / 'metadata' / 'keys.mat'
temp_df = load_conditions_coding(coding_path)
mappping_subject = dict(zip(temp_df.subject_cond, temp_df["T"]))

in order to properly include slope and offset as mediators on the ITPC, and not as main observables, it would be good to add to the spreadsheets you sent the ITPC value per subject, ROI, condition. Since we need a single value, it can be the value averaged in the intervals \[-500,-200\] for prestimuali and  \[400 700\]ms after sound onset, 

Data wasn't z scored or baseline normalized.

In [3]:


def plot_itpc(temp):
    fig, ax = plt.subplots(figsize=(20, 5))
    n_channels = temp['scout_names'].shape[0]
    ax.set_prop_cycle(color=plt.cm.viridis(np.linspace(0, 1, n_channels)))
    temp.plot.line(x='time', hue='scout_names')
    ax.get_legend().remove()



In [4]:
pickle_path = root / "data"/ "share" / "ITPC"/ "list_itpc.pickle"
file = open(pickle_path,'rb')
itpc_list  = pickle.load(file)
file.close()
itpc_list = itpc_list[:1]

def average_itpc(xitpc, tmin, tmax):
    xitpc_stimuli = xitpc.sel(time=slice(tmin, tmax))
    mean_itpc =  xitpc_stimuli.mean(dim='time')
    return mean_itpc


list_xdf = []
for i, item_itpc  in enumerate(itpc_list):
    xdf = pd.DataFrame() 
    time, scout_names = item_itpc.indexes.values()
    xdf['ROI']  =  scout_names
    xdf['subject'] = item_itpc.attrs["subject"]
    xdf['subject_cond'] = item_itpc.attrs["subject_cond"]
    xdf['itpc_prestim'] = average_itpc(item_itpc, -0.5, -0.2).values
    xdf['itpc_stim'] = average_itpc(item_itpc, 0.3, 0.7).values
    list_xdf.append(xdf)
    #sns.scatterplot(data=xdf, x="itpc_prestim", y="itpc_stim")
   
    
df = pd.concat(list_xdf )

FileNotFoundError: [Errno 2] No such file or directory: '/Users/daniel/PhD/Projects/meg-pipeline/data/share/ITPC/list_itpc.pickle'

In [256]:
pickle_path = root / "data"/ "share" / "ITPC"/ "list_itpc.pickle"
file = open(pickle_path,'rb')
itpc_list  = pickle.load(file)
file.close()

def average_itpc(xitpc, tmin, tmax):
    xitpc_stimuli = xitpc.sel(time=slice(tmin, tmax))
    mean_itpc =  xitpc_stimuli.mean(dim='time')
    return mean_itpc

S = ['PRESTIM','STIM' ]
times = [[-0.5, -0.2],[0.4,0.7]  ]
list_xdf = []
for i, item_itpc  in enumerate(itpc_list):
    for i, (cond, interval) in enumerate(zip(S,times)):
        print(i, cond,interval, item_itpc.attrs["subject"])
        xdf = pd.DataFrame()
        time, scout_names = item_itpc.indexes.values()
        xdf['ROI']  =  scout_names
        xdf['subject'] = item_itpc.attrs["subject"]
        xdf['subject_cond'] = item_itpc.attrs["subject_cond"]
        xdf['S'] = cond
        xdf["avg_ITPC"] =  average_itpc(item_itpc, *interval ).values
        list_xdf.append(xdf)

df = pd.concat(list_xdf )
display(df)

0 PRESTIM [-0.5, -0.2] Subject01_A
1 STIM [0.4, 0.7] Subject01_A
0 PRESTIM [-0.5, -0.2] Subject01_A
1 STIM [0.4, 0.7] Subject01_A
0 PRESTIM [-0.5, -0.2] Subject01_B
1 STIM [0.4, 0.7] Subject01_B
0 PRESTIM [-0.5, -0.2] Subject01_B
1 STIM [0.4, 0.7] Subject01_B
0 PRESTIM [-0.5, -0.2] Subject02_A
1 STIM [0.4, 0.7] Subject02_A
0 PRESTIM [-0.5, -0.2] Subject02_A
1 STIM [0.4, 0.7] Subject02_A
0 PRESTIM [-0.5, -0.2] Subject02_B
1 STIM [0.4, 0.7] Subject02_B
0 PRESTIM [-0.5, -0.2] Subject02_B
1 STIM [0.4, 0.7] Subject02_B
0 PRESTIM [-0.5, -0.2] Subject03_A
1 STIM [0.4, 0.7] Subject03_A
0 PRESTIM [-0.5, -0.2] Subject03_A
1 STIM [0.4, 0.7] Subject03_A
0 PRESTIM [-0.5, -0.2] Subject03_B
1 STIM [0.4, 0.7] Subject03_B
0 PRESTIM [-0.5, -0.2] Subject03_B
1 STIM [0.4, 0.7] Subject03_B
0 PRESTIM [-0.5, -0.2] Subject04_A
1 STIM [0.4, 0.7] Subject04_A
0 PRESTIM [-0.5, -0.2] Subject04_B
1 STIM [0.4, 0.7] Subject04_B
0 PRESTIM [-0.5, -0.2] Subject04_B
1 STIM [0.4, 0.7] Subject04_B
0 PRESTIM [-0.5, -0.2] Su

,ROI,subject,subject_cond,S,avg_ITPC
0,G_Ins_lg_and_S_cent_ins L,Subject01_A,01_resample,PRESTIM,0.048124
1,G_Ins_lg_and_S_cent_ins R,Subject01_A,01_resample,PRESTIM,0.058927
2,G_and_S_cingul-Ant L,Subject01_A,01_resample,PRESTIM,0.088520
3,G_and_S_cingul-Ant R,Subject01_A,01_resample,PRESTIM,0.060184
4,G_and_S_cingul-Mid-Ant L,Subject01_A,01_resample,PRESTIM,0.047460
...,...,...,...,...,...
143,S_temporal_inf R,Subject12_A,02_resample,STIM,0.120757
144,S_temporal_sup L,Subject12_A,02_resample,STIM,0.164517
145,S_temporal_sup R,Subject12_A,02_resample,STIM,0.104779
146,S_temporal_transverse L,Subject12_A,02_resample,STIM,0.280830


# Decoding subjects and conditions

In [257]:
x = {'01_resample':'pre', '03_resample':'pre', '02_resample':'post','04_resample':'post', 
    '01_resample_02':'pre', '03_resample_02':'pre', '02_resample_02':'post','04_resample_02':'post', }
df['P'] = df['subject_cond'].map(x)

df['T'] = df['subject'].map(mappping_subject)
df['subjectID'] = df['subject'].str[:9]

df["ROI"] = df.ROI.str.replace(' ', '_') #trick with replacing
list_of_columns = ['subjectID', 'ROI','S','T', 'P', 'avg_ITPC']
itpc_df = df[list_of_columns]
itpc_df = itpc_df.drop_duplicates()
itpc_df= itpc_df.sort_values(by=['subjectID', 'ROI'])
itpc_df

,subjectID,ROI,S,T,P,avg_ITPC
0,Subject01,G_Ins_lg_and_S_cent_ins_L,PRESTIM,real,pre,0.048124
0,Subject01,G_Ins_lg_and_S_cent_ins_L,STIM,real,pre,0.097607
0,Subject01,G_Ins_lg_and_S_cent_ins_L,PRESTIM,real,post,0.038495
0,Subject01,G_Ins_lg_and_S_cent_ins_L,STIM,real,post,0.079172
0,Subject01,G_Ins_lg_and_S_cent_ins_L,PRESTIM,sham,pre,0.087066
...,...,...,...,...,...,...
147,Subject15,S_temporal_transverse_R,STIM,sham,post,0.466148
147,Subject15,S_temporal_transverse_R,PRESTIM,real,pre,0.059402
147,Subject15,S_temporal_transverse_R,STIM,real,pre,0.585000
147,Subject15,S_temporal_transverse_R,PRESTIM,real,post,0.088559


# Import fooof results

In [258]:
root= get_project_root()
fooof_dataframe = root / 'output' / 'share' / 'ROI_offset_exponent_dataframe.csv'
df_fooof = pd.read_csv(fooof_dataframe)
df_fooof["subjectID"]= df_fooof["subjectID"].str.replace(' ', '')
df_fooof

,subjectID,S,T,P,ROI,offset,exponent,errors,r2s
0,Subject01,STIM,sham,pre,G_Ins_lg_and_S_cent_ins_L,-22.116248,1.181624,0.023000,0.994990
1,Subject01,PRESTIM,sham,post,G_Ins_lg_and_S_cent_ins_L,-22.200775,1.129508,0.036272,0.988568
2,Subject01,STIM,sham,post,G_Ins_lg_and_S_cent_ins_L,-22.210722,1.118145,0.041162,0.987425
3,Subject01,PRESTIM,sham,pre,G_Ins_lg_and_S_cent_ins_L,-22.117062,1.192250,0.024770,0.994684
4,Subject01,STIM,real,pre,G_Ins_lg_and_S_cent_ins_L,-22.208380,1.105933,0.030034,0.987846
...,...,...,...,...,...,...,...,...,...
16544,Subject15,PRESTIM,sham,post,S_temporal_transverse_R,-20.867509,1.264356,0.052591,0.987377
16545,Subject15,PRESTIM,real,pre,S_temporal_transverse_R,-20.843362,1.356704,0.071445,0.983309
16546,Subject15,PRESTIM,real,post,S_temporal_transverse_R,-20.832404,1.328099,0.069815,0.982729
16547,Subject15,STIM,real,pre,S_temporal_transverse_R,-20.841836,1.344151,0.073663,0.977591


# Combine results together

In [259]:
result = df_fooof.merge(itpc_df,
               on= ['subjectID', 'S','T','P','ROI'],
                 how='outer'
                      )
result

,subjectID,S,T,P,ROI,offset,exponent,errors,r2s,avg_ITPC
0,Subject01,STIM,sham,pre,G_Ins_lg_and_S_cent_ins_L,-22.116248,1.181624,0.023000,0.994990,0.119523
1,Subject01,PRESTIM,sham,post,G_Ins_lg_and_S_cent_ins_L,-22.200775,1.129508,0.036272,0.988568,0.090768
2,Subject01,STIM,sham,post,G_Ins_lg_and_S_cent_ins_L,-22.210722,1.118145,0.041162,0.987425,0.107943
3,Subject01,PRESTIM,sham,pre,G_Ins_lg_and_S_cent_ins_L,-22.117062,1.192250,0.024770,0.994684,0.087066
4,Subject01,STIM,real,pre,G_Ins_lg_and_S_cent_ins_L,-22.208380,1.105933,0.030034,0.987846,0.097607
...,...,...,...,...,...,...,...,...,...,...
16563,Subject14,STIM,real,post,G_rectus_L,NaN,NaN,NaN,NaN,0.054062
16564,Subject14,STIM,real,post,G_temp_sup-Plan_polar_L,NaN,NaN,NaN,NaN,0.176161
16565,Subject14,PRESTIM,real,post,Pole_temporal_L,NaN,NaN,NaN,NaN,0.066185
16566,Subject14,STIM,real,pre,S_oc_middle_and_Lunatus_L,NaN,NaN,NaN,NaN,0.067291


Rows that contains NaN (Fooof  datframe doesn't containt these row)

In [260]:
df1 = result[result.isna().any(axis=1)]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df1)

,subjectID,S,T,P,ROI,offset,exponent,errors,r2s,avg_ITPC
16549,Subject03,STIM,real,pre,G_rectus_L,NaN,NaN,NaN,NaN,0.078692
16550,Subject03,STIM,real,pre,G_rectus_R,NaN,NaN,NaN,NaN,0.065438
16551,Subject03,PRESTIM,real,post,G_rectus_R,NaN,NaN,NaN,NaN,0.042156
16552,Subject03,PRESTIM,real,pre,S_orbital-H_Shaped_R,NaN,NaN,NaN,NaN,0.082557
16553,Subject04,STIM,real,post,G_and_S_frontomargin_L,NaN,NaN,NaN,NaN,0.072327
16554,Subject04,PRESTIM,real,pre,G_and_S_frontomargin_R,NaN,NaN,NaN,NaN,0.054120
16555,Subject08,STIM,real,post,G_and_S_transv_frontopol_R,NaN,NaN,NaN,NaN,0.047015
16556,Subject08,STIM,sham,post,G_rectus_R,NaN,NaN,NaN,NaN,0.064425
16557,Subject09,PRESTIM,sham,post,Pole_temporal_R,NaN,NaN,NaN,NaN,0.069545
16558,Subject10,PRESTIM,sham,post,G_and_S_frontomargin_R,NaN,NaN,NaN,NaN,0.050551


# Save

In [262]:
result[['subjectID', 'ROI','S','T', 'P']] = result[["subjectID", 'ROI','S','T', 'P']].astype("category")

In [263]:
csv_path = root / 'output' / 'share' / 'ROI_itpc_dataframe.csv'

result.to_csv(csv_path, index = False, header=True)